### **CX arrears analysis**

In [ ]:
%%sh
http_proxy=http://10.245.5.249:8080
export http_proxy
https_proxy=https://10.245.5.249:8080
export https_proxy

pip install gcsfs
pip install factor_analyzer
pip install hdbscan
# pip install numpy==1.16.1
pip install seaborn --upgrade

In [1]:
import csv
import math
import warnings
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from scipy import stats
plt.style.use('fivethirtyeight')
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:.2f}'.format

In [2]:
from sklearn.decomposition import PCA 
from sklearn.cluster import AgglomerativeClustering 
from sklearn.preprocessing import StandardScaler, normalize 
from sklearn.metrics import silhouette_score 
import scipy.cluster.hierarchy as shc

In [3]:
df_ads_v1 = pd.read_csv('gs://aes-datahub-0002-customerservice-curated/CovidTest/res_cust_v8.1_07202020.csv', index_col=0)
print(df_ads_v1.shape)

(239200, 35)


In [4]:
df_ads_v1.head()

,BALANCE,SERVICE_CENTER,Acc_Disconnect_Count,Acc_Letter_Count,Acc_Due_Count_1,Acc_Due_Count_2,Acc_Due_Count_3,Acc_Personal_Contact_count,Acc_Due_Count,CustAcc_Flag,Covid,Onset,precovid,Pa_Opted_Flag,BILL_UNIT_1,BILL_UNIT_2,BILL_UNIT_3,BILL_UNIT_4,BILL_UNIT_5,BILL_UNIT_6,BILL_UNIT_7,BILL_UNIT_8,BILL_UNIT_9,BILL_UNIT_10,BILL_UNIT_11,BILL_UNIT_12,BILL_UNIT_13,BILL_UNIT_14,BILL_UNIT_15,BILL_UNIT_16,BILL_UNIT_17,BILL_UNIT_18,BILL_UNIT_19,BILL_UNIT_20,BILL_UNIT_21
0,75.00,WLM,0,0,2,0,0,0,2,SingleAcc,0,0,2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
1,8.32,WLM,0,0,0,0,0,0,0,MultiAcc,0,0,0,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
2,8.49,WLM,0,0,0,0,0,0,0,MultiAcc,0,0,0,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
3,9.45,WLM,0,0,0,0,0,0,0,MultiAcc,0,0,0,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
4,9.88,WLM,0,0,0,0,0,0,0,MultiAcc,0,0,0,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False


In [5]:
df_custacc_flag = pd.get_dummies(df_ads_v1['CustAcc_Flag'], prefix='CustAcc_Flag', prefix_sep='_')
df_custacc_flag.head()

,CustAcc_Flag_MultiAcc,CustAcc_Flag_SingleAcc
0,0,1
1,1,0
2,1,0
3,1,0
4,1,0


In [6]:
df_custacc_flag['CustAcc_Flag_MultiAcc'] = df_ads_v1['CustAcc_Flag_MultiAcc'].apply(lambda x: 'True' if (x==1) else 'False')
df_custacc_flag['CustAcc_Flag_SingleAcc'] = df_ads_v1['CustAcc_Flag_MultiAcc'].apply(lambda x: 'True' if (x==1) else 'False')
df_custacc_flag.head()

,CustAcc_Flag_MultiAcc,CustAcc_Flag_SingleAcc
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False


In [ ]:
df_custacc_flag['SERVICE_CENTER']

In [7]:
df_ads_v1.drop(['CustAcc_Flag'], axis=1, inplace=True)
df_ads_v2 = pd.merge(df_ads_v1, df_custacc_flag, left_index=True, right_index=True)

In [8]:
df_ads_v2.head()

,BALANCE,SERVICE_CENTER,Acc_Disconnect_Count,Acc_Letter_Count,Acc_Due_Count_1,Acc_Due_Count_2,Acc_Due_Count_3,Acc_Personal_Contact_count,Acc_Due_Count,Covid,Onset,precovid,Pa_Opted_Flag,BILL_UNIT_1,BILL_UNIT_2,BILL_UNIT_3,BILL_UNIT_4,BILL_UNIT_5,BILL_UNIT_6,BILL_UNIT_7,BILL_UNIT_8,BILL_UNIT_9,BILL_UNIT_10,BILL_UNIT_11,BILL_UNIT_12,BILL_UNIT_13,BILL_UNIT_14,BILL_UNIT_15,BILL_UNIT_16,BILL_UNIT_17,BILL_UNIT_18,BILL_UNIT_19,BILL_UNIT_20,BILL_UNIT_21,CustAcc_Flag_MultiAcc,CustAcc_Flag_SingleAcc
0,75.00,WLM,0,0,2,0,0,0,2,0,0,2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
1,8.32,WLM,0,0,0,0,0,0,0,0,0,0,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,8.49,WLM,0,0,0,0,0,0,0,0,0,0,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,9.45,WLM,0,0,0,0,0,0,0,0,0,0,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,9.88,WLM,0,0,0,0,0,0,0,0,0,0,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [9]:
print(df_ads_v2.isnull().sum())
# df_ws.drop(['timestamp'],axis=1,inplace=True)

BALANCE                       0
SERVICE_CENTER                0
Acc_Disconnect_Count          0
Acc_Letter_Count              0
Acc_Due_Count_1               0
Acc_Due_Count_2               0
Acc_Due_Count_3               0
Acc_Personal_Contact_count    0
Acc_Due_Count                 0
Covid                         0
Onset                         0
precovid                      0
Pa_Opted_Flag                 0
BILL_UNIT_1                   0
BILL_UNIT_2                   0
BILL_UNIT_3                   0
BILL_UNIT_4                   0
BILL_UNIT_5                   0
BILL_UNIT_6                   0
BILL_UNIT_7                   0
BILL_UNIT_8                   0
BILL_UNIT_9                   0
BILL_UNIT_10                  0
BILL_UNIT_11                  0
BILL_UNIT_12                  0
BILL_UNIT_13                  0
BILL_UNIT_14                  0
BILL_UNIT_15                  0
BILL_UNIT_16                  0
BILL_UNIT_17                  0
BILL_UNIT_18                  0
BILL_UNI

In [10]:
def Diff(li1, li2): 
    return (list(set(li1) - set(li2))) 
  
numeric_cols = list(df_ads_v2.select_dtypes(include=[np.number]).columns)
all_cols = list(df_ads_v2.columns)
print(Diff(all_cols, numeric_cols))

['BILL_UNIT_8', 'BILL_UNIT_3', 'BILL_UNIT_9', 'BILL_UNIT_2', 'BILL_UNIT_10', 'BILL_UNIT_4', 'SERVICE_CENTER', 'BILL_UNIT_6', 'BILL_UNIT_7', 'BILL_UNIT_1', 'BILL_UNIT_5', 'BILL_UNIT_17', 'BILL_UNIT_18', 'BILL_UNIT_16', 'BILL_UNIT_12', 'BILL_UNIT_20', 'BILL_UNIT_13', 'BILL_UNIT_21', 'CustAcc_Flag_SingleAcc', 'BILL_UNIT_19', 'Pa_Opted_Flag', 'BILL_UNIT_14', 'BILL_UNIT_11', 'BILL_UNIT_15', 'CustAcc_Flag_MultiAcc']


In [11]:
# Note that `select_dtypes` returns a data frame. We are selecting only the columns
from scipy.stats import zscore
numeric_cols = df_ads_v2.select_dtypes(include=[np.number]).columns
df_cx_zscore = df_ads_v2[numeric_cols].apply(zscore)

In [12]:
extra_cols = df_ads_v2[['BILL_UNIT_10', 'BILL_UNIT_14', 'BILL_UNIT_17', 'BILL_UNIT_1', 'BILL_UNIT_9', 'SERVICE_CENTER',
                        'BILL_UNIT_4', 'BILL_UNIT_19', 'CustAcc_Flag_MultiAcc', 'BILL_UNIT_7', 'Pa_Opted_Flag',
                        'CustAcc_Flag_SingleAcc', 'BILL_UNIT_20', 'BILL_UNIT_8', 'BILL_UNIT_15', 'BILL_UNIT_2',
                        'BILL_UNIT_12', 'BILL_UNIT_5', 'BILL_UNIT_18', 'BILL_UNIT_3', 'BILL_UNIT_16', 'BILL_UNIT_6',
                        'BILL_UNIT_13', 'BILL_UNIT_11', 'BILL_UNIT_21']]

In [13]:
df_cx_zscore = pd.merge(df_cx_zscore, extra_cols, left_index=True, right_index=True)
df_cx_zscore.head()

,BALANCE,Acc_Disconnect_Count,Acc_Letter_Count,Acc_Due_Count_1,Acc_Due_Count_2,Acc_Due_Count_3,Acc_Personal_Contact_count,Acc_Due_Count,Covid,Onset,precovid,BILL_UNIT_10,BILL_UNIT_14,BILL_UNIT_17,BILL_UNIT_1,BILL_UNIT_9,SERVICE_CENTER,BILL_UNIT_4,BILL_UNIT_19,CustAcc_Flag_MultiAcc,BILL_UNIT_7,Pa_Opted_Flag,CustAcc_Flag_SingleAcc,BILL_UNIT_20,BILL_UNIT_8,BILL_UNIT_15,BILL_UNIT_2,BILL_UNIT_12,BILL_UNIT_5,BILL_UNIT_18,BILL_UNIT_3,BILL_UNIT_16,BILL_UNIT_6,BILL_UNIT_13,BILL_UNIT_11,BILL_UNIT_21
0,-0.28,-0.16,-0.48,1.40,-0.60,-0.53,-0.02,-0.16,-0.63,-0.63,-0.39,False,False,False,False,False,WLM,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
1,-0.45,-0.16,-0.48,-0.36,-0.60,-0.53,-0.02,-0.76,-0.63,-0.63,-0.64,False,False,False,False,False,WLM,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
2,-0.45,-0.16,-0.48,-0.36,-0.60,-0.53,-0.02,-0.76,-0.63,-0.63,-0.64,False,False,False,False,False,WLM,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
3,-0.45,-0.16,-0.48,-0.36,-0.60,-0.53,-0.02,-0.76,-0.63,-0.63,-0.64,False,False,False,False,False,WLM,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
4,-0.45,-0.16,-0.48,-0.36,-0.60,-0.53,-0.02,-0.76,-0.63,-0.63,-0.64,False,False,False,False,False,WLM,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
